In [ ]:
import json
import pandas as pd
import numpy as np

# Extract

#### Load in the wikipedia JSON

In [ ]:
#file_dir = 'C://Users/Username/DataBootcamp/'
file_dir = 'C://Users/Nat/DataAnalyticsBootcamp/Module_8/Movies_ETL/Resources/'

In [ ]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)
    
wiki_movies_raw

In [ ]:
len(wiki_movies_raw)

##### Inspect the data

In [ ]:
# First 5 records
wiki_movies_raw[:5]

# Last 5 records
wiki_movies_raw[-5:]

# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

#### Load in Kaggle Data from MovieLens

In [ ]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

##### Inspect the data

In [ ]:
kaggle_metadata.sample(10)
ratings.sample(10)

# Transform

### Filtering Bad Data (ie tv show data )

In [ ]:
wiki_movies_df.sample(20)

In [ ]:
# See the column names of all 193 columns
wiki_movies_df.columns.tolist()

In [ ]:
# Let's modify our JSON data by restricting it to only those entries that have a director and an IMDb link
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.sample(10)
# now there is only 78 columns of data

In [ ]:
# See the column names of all 78 columns
wiki_movies_df.columns.tolist()

In [ ]:
# It looks like we've got some TV shows in our data instead of movies.
# Delete rows with 'No. of episodes' data
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.sample(10)
# now there is only 75 columns of data

### Clean the alternative titles (ie titles in other language columns)

#### Make a function called clean_movie to clean up each movie entry

In [ ]:
# The different language columns are for alternate titles of the movie. 
# Let's combine all of them into one dictionary that has all the alternate titles.

# Step 1 Make an Empty dictionary to hold all alternative titles
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    return movie

In [35]:
# Step 2a: Loop through a list of all alternative title keys.
#          Check if the current key exists in the movie object.
# Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
            
    return movie

In [36]:
# Step 3: After looping through every key, add the alternative titles dict to the movie object.
def clean_movie(movie):
    
    movie = dict(movie) #create a non-destructive copy
    
    alt_titles = {}
    
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
            
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

#### Use the function you created to make a list of cleaned movies

In [ ]:
# make a list of cleaned movies
clean_movies = [clean_movie(movie) for movie in wiki_movies]

#### Update the dataframe with clean data

In [37]:
# Make a new wiki_movies_df based on data in clean movies 
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_movies_df.sample(5)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Released,Recorded,Venue,Length,Label,Director,Producer,Animation by,Color process,Screen story by
664,https://en.wikipedia.org/wiki/Where_the_Day_Ta...,1992,https://www.imdb.com/title/tt0105810/,Where the Day Takes You,Marc Rocco,"[Paul Hertzberg, Philip McKeon]",NaN,NaN,NaN,"[Sean Astin, Lara Flynn Boyle, Peter Dobson, B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3336,https://en.wikipedia.org/wiki/It_Runs_in_the_F...,2003,https://www.imdb.com/title/tt0311110/,It Runs in the Family,Fred Schepisi,"[Michael Douglas, Marcy Drogin, Jesse Wigutow]",NaN,NaN,NaN,"[Michael Douglas, Kirk Douglas, Cameron Dougla...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,https://en.wikipedia.org/wiki/Across_the_Tracks,1991,https://www.imdb.com/title/tt0101268/,Across the Tracks,Sandy Tung,"[Francesca Bill, Nancy Paloian, Dale Rosenbloo...",NaN,NaN,NaN,"[Rick Schroder, Brad Pitt, Carrie Snodgress, D...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4043,https://en.wikipedia.org/wiki/Fay_Grim,2006,https://www.imdb.com/title/tt0444628/,Fay Grim,Hal Hartley,"[Hal Hartley, Jason Kilot, Joana Vicente]",NaN,NaN,NaN,"[Parker Posey, James Urbaniak, Liam Aiken, Jef...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3898,https://en.wikipedia.org/wiki/White_Noise_(film),2005,https://www.imdb.com/title/tt0375210/,White Noise,Geoffrey Sax,Paul Brooks,NaN,NaN,NaN,"[Michael Keaton, Deborah Kara Unger, Mike Dopu...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# display the list of all 56 columns
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

### Clean columns with oh-so-slightly different names

In [40]:
# We need write a function to consolidate columns with the same data into one column 
# This will go inside of the clean_movie function that we already created

In [41]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [44]:
# Make a new wiki_movies_df based on data in clean movies & changed col names 
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_movies_df.sample(5)

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Created by,Preceded by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer
358,https://en.wikipedia.org/wiki/Kafka_(film),1991,https://www.imdb.com/title/tt0102181/,Kafka,NaN,"[Jeremy Irons, Theresa Russell, Joel Grey, Ian...",NaN,Walt Lloyd,"[November 15, 1991, (, 1991-11-15, )]",98 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5902,https://en.wikipedia.org/wiki/Indescribable_(f...,2013,https://www.imdb.com/title/tt2385111/,Indescribable,NaN,"[Seth Pruski, Rich Swingle, Jason Cockerham, D...",NaN,"[Timothy Jones, Ethan Ledden]","April 6, 2013",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
706,https://en.wikipedia.org/wiki/Coneheads_(film),1993,https://www.imdb.com/title/tt0106598/,Coneheads,"[Coneheads, sketches from, Saturday Night Live...","[Dan Aykroyd, Jane Curtin]",NaN,Francis Kenny,"[July 23, 1993, (, 1993-07-23, )]",87 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5955,https://en.wikipedia.org/wiki/Manhattan_Romance,2013,https://www.imdb.com/title/tt2608324/,Manhattan Romance,NaN,"[Gaby Hoffmann, Katherine Waterston, Zach Gren...",NaN,Scott Miller,"[February 2, 2015, (, 2015-02-02, ), (United S...",94 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,https://en.wikipedia.org/wiki/Cool_as_Ice,1991,https://www.imdb.com/title/tt0101615/,Cool as Ice,NaN,"[Vanilla Ice, Kristin Minter, Michael Gross]",NaN,Janusz Kamiński,"[October 18, 1991, (, 1991-10-18, )]",91 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# display the list of all 40 columns
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'McCune–Reischauer',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']